<table>
<tr>
  <td style="vertical-align: top; padding-right: 20px;">

На планете **СтайлГания-II** государство ввело инновационную систему оформления паспортов: вместо фотографирования граждан, правительство делает портреты на основании «генетического кода» каждого человека. Специальный модуль — биометрический конвейер — берет ДНК-строку `z`, анализирует и синтезирует уникальное изображение лица в высоком разрешении.

Недавно появились новые законодательные требования:

- В паспорте запрещена борода — даже если человек ее носит в жизни.

- В паспорте обязательно улыбаться — лицо должно быть веселым.

Денег на изменения системы у правительства не осталось, все ушло на подготовку самых перспективных представителей к межпланетной олимпиаде по ИИ.

Предложите решение, как можно попробовать решить эту проблему, например - изменив входной ген `z` нужным образом, чтобы в финальном изображении не было бороды, а грустный представитель планеты обязательно улыбался, сохраняя при этом все остальные черты личности (возраст, цвет волос, форма лица и т.п.).

Правительство любит вводить новые требования, поэтому задумайтесь заранее, чтобы ваше решение можно было адаптировать под новые ограничения на фото.

  </td>
  <td style="vertical-align: top;">

<img src="SG2.png" width="500"/>

  </td>
</tr>
</table>


Заберем всю необходимую информацию для работы с системой.

In [ ]:
import os

backend = 'Colab'

if backend == 'Colab':
    # Клонируем репозиторий, если ещё нет
    if not os.path.isdir('/content/GANs-CU-2025'):
        !git clone --depth 1 https://github.com/airogachev/GANs-CU-2025.git

    # Обновляем apt и устанавливаем ninja
    !sudo apt-get update -qq
    !sudo apt-get install -y ninja-build

    # Переходим в нужную папку
    %cd /content/GANs-CU-2025/HW


Получим опытный образец рабочей модели визуализации паспортов.

In [ ]:
import os
import sys

# 1. Скачиваем pretrained.tar только если ещё нет файла
if not os.path.isfile('pretrained.tar'):
    !wget -q --show-progress https://www.dropbox.com/s/2kpsomtla61gjrn/pretrained.tar -O pretrained.tar
else:
    print('pretrained.tar уже загружен')

# 2. Распаковываем в папку pretrained/, если её нет
if not os.path.isdir('pretrained'):
    !tar -xvf pretrained.tar
else:
    print('Каталог pretrained/ уже существует')

# 3. Добавляем в PYTHONPATH папку с кодом
module_path = '/content/GANs-CU-2025/HW'
if module_path not in sys.path:
    sys.path.append(module_path)
    print(f'Добавил {module_path} в sys.path')
else:
    print(f'{module_path} уже в sys.path')

Подготовим среду для экспериментов.

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision.utils import make_grid
from torchvision.transforms import ToPILImage
from tqdm.auto import tqdm, trange
from PIL import Image

from gans.gan_load import make_stylegan2

# ---------------- Настройки окружения ----------------
# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"CUDA devices count: {torch.cuda.device_count()}")
print(f"PyTorch version: {torch.__version__}")

# Семена для воспроизводимости при работе с генами
torch.manual_seed(42)
np.random.seed(42)

# -----------------------------------------------------

def to_image(tensor: torch.Tensor, adaptive: bool = False) -> Image.Image:
    """
    Преобразует тензор StyleGAN (в диапазоне [-1,1]) в PIL.Image.
    Если tensor.ndim==4, берёт первый канал по batch.
    Если adaptive=True — масштабирует по собственным мин/макс;
    иначе линейно переводит из [-1,1] в [0,1].
    """
    if tensor.ndim == 4:
        tensor = tensor[0]
    tensor = tensor.cpu().detach()
    if adaptive:
        tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min())
    else:
        tensor = ((tensor + 1) / 2).clamp(0, 1)
    img_uint8 = (tensor * 255).to(torch.uint8)
    return ToPILImage()(img_uint8)

def to_image_grid(tensors: torch.Tensor, adaptive: bool = False, nrow: int = 8, padding: int = 2) -> Image.Image:
    """
    Создаёт сетку из batch-тензоров [B, C, H, W] и возвращает PIL.Image.
    Параметры nrow, padding передаются в torchvision.utils.make_grid.
    """
    grid = make_grid(tensors, nrow=nrow, padding=padding)
    return to_image(grid, adaptive)


Запустим опытный образец, который по случайному стечению обстоятельств именнуется StyleGAN2

In [ ]:
from pathlib import Path
import torch

# ---------------- Настройки модели ----------------
# Устройство (повторно определяем для надёжности)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Путь к чекпоинту (относительно текущей директории)
CKPT_PATH = Path('pretrained/stylegan2-ffhq-config-f.pt')
assert CKPT_PATH.exists(), f"❌ Checkpoint not found: {CKPT_PATH}"

# Загружаем StyleGAN2
# В pretrained.tar обычно лежит g_ema, поэтому target_key='g_ema'
G = make_stylegan2(
    resolution=1024,
    weights=str(CKPT_PATH),
    target_key='g_ema',     # или None, если ключи в state_dict сразу «g»
    shift_in_w=True
).to(device)
G.eval()

# Размерность гена, который используется для генерации
LATENT_DIM = getattr(G, 'dim_z', 512)
print(f"Loaded StyleGAN2 generator — latent dim = {LATENT_DIM}")


Посмотрим, кто живет на этой планете и как пользоваться поделием для визуализации.

In [ ]:
# Простая генерация 12 случайных лиц
with torch.no_grad():
    z = torch.randn(12, LATENT_DIM, device=device)
    imgs = G(z)  # plain generation без truncation

plt.figure(figsize=(8,8), dpi=150)
plt.axis('off')
plt.imshow(to_image_grid(imgs, nrow=4))
plt.show()

Посмотрим, дозволено ли нам заниматься арифметикой генов.

In [ ]:
with torch.no_grad():
    # 1) Генерируем два разных z
    z1 = torch.randn(1, LATENT_DIM, device=device)
    z2 = torch.randn(1, LATENT_DIM, device=device)
    # 2) Простая генерация «до»
    img1 = G(z1)                   # лицо из z1
    img2 = G(z2)                   # лицо из z2

    # 3) Переводим в w-пространство
    w1 = G.style_gan2.get_latent(z1).unsqueeze(1).repeat(1, G.style_gan2.n_latent, 1)
    w2 = G.style_gan2.get_latent(z2).unsqueeze(1).repeat(1, G.style_gan2.n_latent, 1)
    # 4) Собираем микс:  первые 6 слоёв из w1, остальные из w2
    w_mix = w1.clone()
    w_mix[:, 6:] = w2[:, 6:]
    img_mix = G(w_mix, w_space=True)  # смешанное лицо

imgs = torch.cat([img1, img2, img_mix], dim=0)
plt.figure(figsize=(9,3), dpi=150)
plt.axis('off')
plt.imshow(to_image_grid(imgs, nrow=3))
plt.show()


Решите главную задачу, поставленную руководством. Принимайте самые смелые решения для контроля визуализации, но не меняйте исходную модель. Не забудьте постараться по возможности сделать решение масштабируемым на новые требования к фото на паспорт.